# Functions to generate frame states

In [1]:
# Although I have made some minor modifications, this part of the code was given to me by Peter Xenopoulos
import lzma
import json

# Function to read .xz archives from ESTA
def read_parsed_demo(filename):
  with lzma.LZMAFile(filename, "rb") as f:
    d = json.load(f)
    return d

def generate_vector_state(frame, map_name):
    """Returns a game state in a dictionary format.

    Args:
        frame (dict) : Dict output of a frame generated from the DemoParser class
        map_name (string): String indicating the map name

    Returns:
        A dict with keys for each feature.
    """
    game_state = {}
    game_state["mapName"] = map_name
    game_state["secondsSincePhaseStart"] = frame["seconds"]
    game_state["bombPlanted"] = frame["bombPlanted"]
    game_state["bombsite"] = frame["bombsite"]
    game_state["totalSmokes"] = len(frame["smokes"])
    game_state["totalFires"] = len(frame["fires"])

    # Team specific info (CT)
    game_state["ctAlive"] = 0
    game_state["ctHp"] = 0
    game_state["ctArmor"] = 0
    game_state["ctHelmet"] = 0
    game_state["ctEq"] = 0
    game_state["ctUtility"] = 0
    game_state["ctEqValStart"] = 0
    game_state["ctBombZone"] = 0
    game_state["defusers"] = 0
    game_state["ctNone"] = 0
    game_state["ctCash"] = 0
    if frame["ct"]["players"] != None:
        for p in frame["ct"]["players"]:
            game_state["ctEqValStart"] += p["equipmentValueFreezetimeEnd"]
            game_state["ctCash"] += p["cash"]
            if p["isAlive"]:
                game_state["ctAlive"] += 1
                game_state["ctHp"] += p["hp"]
                game_state["ctArmor"] += p["armor"]
                game_state["ctHelmet"] += p["hasHelmet"]
                game_state["ctEq"] += p["equipmentValue"]
                game_state["ctUtility"] += p["totalUtility"]
                game_state["defusers"] += p["hasDefuse"]
                if p["isInBombZone"]:
                    game_state["ctBombZone"] += 1
    else: 
        game_state["ctNone"] = 1

    # Team specific info (T)
    game_state["tAlive"] = 0
    game_state["tHp"] = 0
    game_state["tArmor"] = 0
    game_state["tHelmet"] = 0
    game_state["tEq"] = 0
    game_state["tUtility"] = 0
    game_state["tEqValStart"] = 0
    game_state["tHoldingBomb"] = 0
    game_state["tBombZone"] = 0
    game_state["tNone"] = 0
    game_state["tCash"] = 0
    if frame["t"]["players"] != None:
        for p in frame["t"]["players"]:
            game_state["tEqValStart"] += p["equipmentValueFreezetimeEnd"]
            game_state["tCash"] += p["cash"]
            if p["isAlive"]:
                game_state["tAlive"] += 1
                game_state["tHp"] += p["hp"]
                game_state["tArmor"] += p["armor"]
                game_state["tHelmet"] += p["hasHelmet"]
                game_state["tEq"] += p["equipmentValue"]
                game_state["tUtility"] += p["totalUtility"]
                if p["isInBombZone"]:
                    game_state["tBombZone"] += 1
                if p["hasBomb"]:
                    game_state["tHoldingBomb"] = 1
    else: 
        game_state["tNone"] = 1

    return game_state

# Via json files

In [2]:
# Folder where demos are stored
#path_to_demos = "C:\\Users\\Matias\\Downloads\\esta\\data\\online"
path_to_demos = "..\\input"
path_to_json = "..\\jsons"
# Folder to output dataframe
output = "..\\output"
# Folder with txt file for demos to view
path_to_txt = "..\\Demos_a_revisar.txt"

In [3]:
'''
Parsing games, compressing json file, compressing it and deleting .dem files
'''
from openpyxl import Workbook
from awpy.parser import DemoParser
import pandas as pd
import os, shutil
import patoolib
import zipfile
from zipfile import ZipFile

os.chdir(path_to_demos)

# Main dataframe
round_state_df = pd.DataFrame()

carpetas = []
for file in os.listdir():
    if ".rar" in file:
        patoolib.extract_archive("%s" % file, outdir="%s" %path_to_demos)

    # Grab demo names
    demos = []
    for demo in os.listdir():
        if (".dem" in demo) and (demo[0]!="x"):
            os.rename(demo, "x" + file.split("vs")[0] + "vs" + demo.split("vs")[1])
            demos += ["x" + file.split("vs")[0] + "vs" + demo.split("vs")[1]]
        if ".json" in demo:
            os.remove(demo)
            
    ### Itero parser over demos
    for demo in demos:
        demo_parser = DemoParser(
        demofile = "%s" % demo,
        parse_rate=128, 
        buy_style="hltv",
        parse_chat = True
        )
    
        try:
            # Parse the demofile, output results to dictionary
            df = demo_parser.parse(return_type="json")
    
            print(f"{demo[1:-4]}")
            os.remove(demo)
            shutil.move(f'x{demo[1:-4]}.json', f"{path_to_json}")
            
        except Exception as e:
            print(e)
            continue
    
    os.remove(file)

patool: Extracting blast-premier-fall-groups-2023-complexity-vs-ninjas-in-pyjamas-bo3-WUVPf054ITfrylPmBtp46e.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\blast-premier-fall-groups-2023-complexity-vs-ninjas-in-pyjamas-bo3-WUVPf054ITfrylPmBtp46e.rar"
patool:     with cwd=..\input
patool: ... blast-premier-fall-groups-2023-complexity-vs-ninjas-in-pyjamas-bo3-WUVPf054ITfrylPmBtp46e.rar extracted to `..\input'.
blast-premier-fall-groups-2023-complexity-vs-ninjas-in-pyjamas-m1-mirage
blast-premier-fall-groups-2023-complexity-vs-ninjas-in-pyjamas-m2-overpass
blast-premier-fall-groups-2023-complexity-vs-ninjas-in-pyjamas-m3-nuke
patool: Extracting blast-premier-fall-groups-2023-faze-vs-g2-bo3-ghH83ofICHkweejWhDNwHs.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\blast-premier-fall-groups-2023-faze-vs-g2-bo3-ghH83ofICHkweejWhDNwHs.rar"
patool:     with cwd=..\input
patool

patool: ... blasttv-paris-major-2023-9ine-vs-liquid-overpass-1KPu2iOzCOZki9_tRiiVKT.rar extracted to `..\input'.
blasttv-paris-major-2023-9ine-vs-liquid-inferno
patool: Extracting blasttv-paris-major-2023-americas-rmr-pain-vs-furia-bo3-CDwyXwsl5I91q87THOIbQc.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\blasttv-paris-major-2023-americas-rmr-pain-vs-furia-bo3-CDwyXwsl5I91q87THOIbQc.rar"
patool:     with cwd=..\input
patool: ... blasttv-paris-major-2023-americas-rmr-pain-vs-furia-bo3-CDwyXwsl5I91q87THOIbQc.rar extracted to `..\input'.
blasttv-paris-major-2023-americas-rmr-pain-vs-furia-m1-overpass
blasttv-paris-major-2023-americas-rmr-pain-vs-furia-m2-vertigo
blasttv-paris-major-2023-americas-rmr-pain-vs-furia-m3-mirage
patool: Extracting blasttv-paris-major-2023-apeks-vs-ninjas-in-pyjamas-bo3-sWrCCVh6A5ev0My0Fk0NDC.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\bl

blasttv-paris-major-2023-challengers-stage-forze-vs-liquid-nuke
patool: Extracting blasttv-paris-major-2023-challengers-stage-mouz-vs-og-ancient-qXTVn0En494X3fF2pilY1n.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\blasttv-paris-major-2023-challengers-stage-mouz-vs-og-ancient-qXTVn0En494X3fF2pilY1n.rar"
patool:     with cwd=..\input
patool: ... blasttv-paris-major-2023-challengers-stage-mouz-vs-og-ancient-qXTVn0En494X3fF2pilY1n.rar extracted to `..\input'.
blasttv-paris-major-2023-challengers-stage-mouz-vs-og-ancient
patool: Extracting blasttv-paris-major-2023-challengers-stage-ninjas-in-pyjamas-vs-mouz-mirage-g9o7C9iv5_ZOTfydVJ7QH2.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\blasttv-paris-major-2023-challengers-stage-ninjas-in-pyjamas-vs-mouz-mirage-g9o7C9iv5_ZOTfydVJ7QH2.rar"
patool:     with cwd=..\input
patool: ... blasttv-paris-major-2023-challengers-stage

patool: ... blasttv-paris-major-2023-faze-vs-natus-vincere-bo3-Wi_Mglbs3MZXOsFnzZr0of.rar extracted to `..\input'.
blasttv-paris-major-2023-faze-vs-natus-vincere-m1-overpass
blasttv-paris-major-2023-faze-vs-natus-vincere-m2-mirage
blasttv-paris-major-2023-faze-vs-natus-vincere-m3-anubis
patool: Extracting blasttv-paris-major-2023-fnatic-vs-g2-bo3-8uXeY7wev7x1RLLlhMoGXM.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\blasttv-paris-major-2023-fnatic-vs-g2-bo3-8uXeY7wev7x1RLLlhMoGXM.rar"
patool:     with cwd=..\input
patool: ... blasttv-paris-major-2023-fnatic-vs-g2-bo3-8uXeY7wev7x1RLLlhMoGXM.rar extracted to `..\input'.
blasttv-paris-major-2023-fnatic-vs-g2-m1-inferno
blasttv-paris-major-2023-fnatic-vs-g2-m2-ancient
blasttv-paris-major-2023-fnatic-vs-g2-m3-vertigo
patool: Extracting blasttv-paris-major-2023-fnatic-vs-ninjas-in-pyjamas-nuke-UrO28sLXAWGoVMqHsTKdm6.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:

brazy-party-2023-pain-vs-big-m1-mirage
brazy-party-2023-pain-vs-big-m2-nuke
brazy-party-2023-pain-vs-big-m3-vertigo
patool: Extracting cct-2023-online-finals-1-big-vs-9ine-bo3-vhgG_Rvr5SqUylPlBxIWqu.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\cct-2023-online-finals-1-big-vs-9ine-bo3-vhgG_Rvr5SqUylPlBxIWqu.rar"
patool:     with cwd=..\input
patool: ... cct-2023-online-finals-1-big-vs-9ine-bo3-vhgG_Rvr5SqUylPlBxIWqu.rar extracted to `..\input'.
cct-2023-online-finals-1-big-vs-9ine-m1-anubis
cct-2023-online-finals-1-big-vs-9ine-m2-nuke
patool: Extracting cct-2023-online-finals-1-into-the-breach-vs-big-bo3-_nDsv0cMDF-N_KCU6NiIKq.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\cct-2023-online-finals-1-into-the-breach-vs-big-bo3-_nDsv0cMDF-N_KCU6NiIKq.rar"
patool:     with cwd=..\input
patool: ... cct-2023-online-finals-1-into-the-breach-vs-big-bo3-_nDsv0cMDF-N_KCU6Ni

patool: ... iem-dallas-2023-g2-vs-faze-bo3-eDSx2F5_l0n615JHVYbZlN.rar extracted to `..\input'.
iem-dallas-2023-g2-vs-faze-m1-ancient
iem-dallas-2023-g2-vs-faze-m2-inferno
iem-dallas-2023-g2-vs-faze-m3-mirage
patool: Extracting iem-dallas-2023-g2-vs-heroic-bo3-EHeAlJWWh0-TTx9i81Uvaj.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\iem-dallas-2023-g2-vs-heroic-bo3-EHeAlJWWh0-TTx9i81Uvaj.rar"
patool:     with cwd=..\input
patool: ... iem-dallas-2023-g2-vs-heroic-bo3-EHeAlJWWh0-TTx9i81Uvaj.rar extracted to `..\input'.
iem-dallas-2023-g2-vs-heroic-m1-inferno
iem-dallas-2023-g2-vs-heroic-m2-ancient
patool: Extracting iem-dallas-2023-g2-vs-og-bo3-D73xLF2dVgktRYSurIXB86.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\iem-dallas-2023-g2-vs-og-bo3-D73xLF2dVgktRYSurIXB86.rar"
patool:     with cwd=..\input
patool: ... iem-dallas-2023-g2-vs-og-bo3-D73xLF2dVgktRYSurIXB86.rar extra

patool: ... iem-rio-2023-ninjas-in-pyjamas-vs-natus-vincere-bo3-4CXxWjLvH0L-gheEN6XdLF.rar extracted to `..\input'.
iem-rio-2023-ninjas-in-pyjamas-vs-natus-vincere-m1-mirage
iem-rio-2023-ninjas-in-pyjamas-vs-natus-vincere-m2-ancient
patool: Extracting iem-rio-2023-og-vs-cloud9-bo3-tDfCX_glCmqF87jj7Af-8m.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\iem-rio-2023-og-vs-cloud9-bo3-tDfCX_glCmqF87jj7Af-8m.rar"
patool:     with cwd=..\input
patool: ... iem-rio-2023-og-vs-cloud9-bo3-tDfCX_glCmqF87jj7Af-8m.rar extracted to `..\input'.
iem-rio-2023-og-vs-cloud9-m1-mirage
iem-rio-2023-og-vs-cloud9-m2-overpass
patool: Extracting iem-rio-2023-og-vs-vitality-bo3.rar ...
patool: running "C:\Program Files\WinRAR\rar.EXE" x -- "C:\Users\Matias\Dropbox (Personal)\MaE\input\iem-rio-2023-og-vs-vitality-bo3.rar"
patool:     with cwd=..\input
patool: ... iem-rio-2023-og-vs-vitality-bo3.rar extracted to `..\input'.
iem-rio-2023-og-vs-vitality-m1

In [5]:
from openpyxl import Workbook
from awpy.parser import DemoParser
import pandas as pd
import os, shutil
import patoolib
import json
import lzma


os.chdir("..\\jsons")

# Main dataframe
round_state_df = pd.DataFrame()

# Grab demo names
demos = []
for demo in os.listdir():
    if ".json" in demo:
        print(f"{demo}")
        with open(demo, encoding="utf-8") as demo_json:
                df = json.load(demo_json)
        #with open(demo, encoding = 'Latin-1') as f:
        #    df = json.load(f)
    # Grab round end frames (last frame of every round)
        frames = []
        iteration = 1
        for ronda in df["gameRounds"]:
            for f in ronda["frames"]:
                if f["clockTime"] == "01:55":
                    frames.append(ronda["frames"][-1])
                    break
        mapa = df["mapName"]
        
    
        # Generate vectors for each frame and pass them to df
        states = []
        for f in frames:
            game_state = generate_vector_state(f, mapa)
            states.append(game_state)
        states = pd.DataFrame(states)
        states["matchID"] = (demo[:-4])
        
        try:
            # Get total freeze time
            freezeTimeEnd = []
            startTick = []
            ctTeam = []
            tTeam = []
            winningSide = []
            for round in df["gameRounds"]:
                freezeTimeEnd.append(round["freezeTimeEndTick"])
                startTick.append(round["startTick"])
                ctTeam.append(round["ctTeam"])
                tTeam.append(round["tTeam"])
                winningSide.append(round["winningSide"])

            states["freezeTimeEndTick"] = freezeTimeEnd
            states["startTick"] = startTick
            states["ctTeam"] = ctTeam
            states["tTeam"] = tTeam
            states["winningSide"] = winningSide
            states["freezeTimeTotal"] = states["freezeTimeEndTick"] - states["startTick"] 
            states["pause"] = 0
            states["player"] = ""
            states["msg"] = ""

            # Iterate over messages, if they include a mention of a pause (either include word "pause" or "tech"), then:
            #    If msgtick >= freezeTimeEndTick ==> pause = 1 in current roundnum + 1

            for msg in df['chatMessages']:

                if msg['params'] is not None:
                    if "tech" in msg["text"] or "pause" in msg["text"]:
                        print(f'There was a pause called by {msg["params"][0]}:  {msg["text"]} at tick {msg["tick"]}' )
                        index = states.index[states["freezeTimeEndTick"]>=msg["tick"]][0]
                        if states["pause"].loc[index] == 0:
                            states.loc[index, "pause"] = 1
                            states.loc[index, "player"] = msg["params"][0]
                            states.loc[index, "msg"] = msg["text"]
                            
                if msg['params'] is None:
                    if "A player disconnected, auto pausing." in msg["text"] or "Waiting for both teams and admin to ready to continue." in msg["text"]:
                        index = states.index[states["freezeTimeEndTick"]>=msg["tick"]][0]
                        if states["pause"].loc[index] == 0:
                            states.loc[index, "pause"] = 1
                            states.loc[index, "player"] = "Admin"
                            states.loc[index, "msg"] = msg["text"]
                        
                            
            for i in range(0,5):
                states[f"ct_p{i+1}"] = ""
                states[f"t_p{i+1}"] = ""   
                states.loc[0:15, f"ct_p{i+1}"] = df["gameRounds"][0]["ctSide"]["players"][i]["playerName"]
                states.loc[0:15, f"t_p{i+1}"]  = df["gameRounds"][0]["tSide"]["players"][i]["playerName"]
                states.loc[15:, f"ct_p{i+1}"]  = df["gameRounds"][0]["tSide"]["players"][i]["playerName"]
                states.loc[15:, f"t_p{i+1}"]  = df["gameRounds"][0]["ctSide"]["players"][i]["playerName"]

        except:
            states["mismatch"] = 0
            
            for msg in df['chatMessages']:

                if msg['params'] is not None:
                    if "tech" in msg["text"] or "pause" in msg["text"]:
                        states.loc[:, "mismatch"] = 1
        
        round_state_df = pd.concat([round_state_df, pd.DataFrame(states)])
        
round_state_df.reset_index(inplace = True)
round_state_df["roundNum"] = round_state_df["index"] + 1
round_state_df.drop("index", axis = 1, inplace = True)

with pd.ExcelWriter(
    f"{output}\\parsed.xlsx",
    mode = "w",
    engine = "xlsxwriter"
) as writer:
    round_state_df.to_excel(writer, index = False, startrow = 85374, header = True)

xblast-premier-fall-groups-2023-complexity-vs-ninjas-in-pyjamas-m1-mirage.json
xblast-premier-fall-groups-2023-complexity-vs-ninjas-in-pyjamas-m2-overpass.json
xblast-premier-fall-groups-2023-complexity-vs-ninjas-in-pyjamas-m3-nuke.json
xblast-premier-fall-groups-2023-faze-vs-g2-m1-nuke.json
There was a pause called by NiKo:  .pause at tick 255891
There was a pause called by NiKo:  !pause at tick 256065
xblast-premier-fall-groups-2023-faze-vs-g2-m2-mirage.json
There was a pause called by ropz:  !pause at tick 170105
xblast-premier-fall-groups-2023-faze-vs-liquid-m1-inferno.json
xblast-premier-fall-groups-2023-faze-vs-liquid-m2-mirage.json
xblast-premier-fall-groups-2023-faze-vs-liquid-m3-ancient.json
xblast-premier-fall-groups-2023-g2-vs-liquid-m1-ancient.json
There was a pause called by HooXi:  !pause at tick 366256
There was a pause called by NiKo:  !pause at tick 366270
xblast-premier-fall-groups-2023-g2-vs-liquid-m2-mirage.json
xblast-premier-fall-groups-2023-g2-vs-liquid-m3-vertig

There was a pause called by Xyp9x:  .tech at tick 96954
There was a pause called by Xyp9x:  !pause at tick 97207
xblasttv-paris-major-2023-europe-rmr-b-astralis-vs-spirit-m3-overpass.json
There was a pause called by dev1ce:  !tech at tick 23613
There was a pause called by dev1ce:  !pause at tick 23969
There was a pause called by blameF:  !pause at tick 24625
There was a pause called by gla1ve:  !tech at tick 24929
There was a pause called by dev1ce:  !pause at tick 25374
There was a pause called by chopper:  !pause at tick 25772
xblasttv-paris-major-2023-europe-rmr-b-big-vs-vitality-m1-mirage.json
There was a pause called by tabseN:  !tech at tick 32457
There was a pause called by tabseN:  !pause at tick 32647
There was a pause called by tabseN:  !pause at tick 37020
xblasttv-paris-major-2023-europe-rmr-b-big-vs-vitality-m2-nuke.json
There was a pause called by apEX:  you called a tech they said at tick 20489
xblasttv-paris-major-2023-europe-rmr-b-big-vs-vitality-m3-overpass.json
There

xesl-challenger-katowice-2023-europe-closed-qualifier-big-vs-ninjas-in-pyjamas-m3-vertigo.json
xesl-challenger-katowice-2023-forze-vs-ence-m1-nuke.json
There was a pause called by r3salt:  .techn at tick 9954
There was a pause called by r3salt:  .tech at tick 10197
There was a pause called by shalfey:  .tech at tick 73118
There was a pause called by r3salt:  .tech at tick 78116
There was a pause called by shalfey:  .tech at tick 78248
xesl-challenger-katowice-2023-forze-vs-ence-m2-overpass.json
xesl-challenger-league-season-45-europe-apeks-vs-monte-m1-ancient.json
xesl-challenger-league-season-45-europe-apeks-vs-monte-m2-mirage.json
xesl-challenger-league-season-45-europe-apeks-vs-monte-m3-anubis.json
xiem-dallas-2023-ence-vs-faze-m1-mirage.json
There was a pause called by dycha:  !pause at tick 10596
There was a pause called by Snappi:  .tech at tick 10709
There was a pause called by Snappi:  .tech at tick 77072
There was a pause called by Snappi:  .tech at tick 93923
xiem-dallas-2023

KeyError: 'Sheet1'

In [ ]:
df["gameRounds"][0]

In [6]:
with pd.ExcelWriter(
    f"{output}\\parsed.xlsx",
    mode = "w",
    engine = "xlsxwriter"
) as writer:
    round_state_df.to_excel(writer, index = False, startrow = 85374, header = True)